# Article Important Links
Algorithms to atribute article links importance scores 

KEEP WORKING ON THIS FORMALIZATION AND COMPILATION OF TECHNICS TO COMPUTE IMPORTANT LINKS
MAYBE EXPERIMENT WIKISYNS BASED ON BACKLINKS
EXPERIMENT VARIATIONS OF WIKISYNS 
COMPUTE WORD IMPORTANCE BY ITS SPREAD IN THE ARTICLE (MAYBE TO THIS IN OTHER NOTEBOOK SINCE THIS ONE IS FOR LINKS)
CREATE TREE WITH DICT SYMS

YOU COULD HAVE A LINK STRUCTURE BASED ON UNSUPERVISIED MODELS AND NLP BASED ON SUPERVISED MODELS
THE LINK STRUCTURE WOULD GIVE A BROAD PATH, MEANING THAT DEEP TRANSVERSAL WOULD MEAN SOMETHING YET, BUT NLP BASED WOULD ONLY BE LOCAL
WE COULD USE WIKISYN TO FIND "POSSIBLE LINKS" THAT WERE NOT CREATED

FOR SUPERVISED WE MAY USE:
    NPM DATA
    WORDNET (AND OTHERS) DICTIONARIES
    SCHOOL AND COURSES CURRICULUMS 

LINKS PROBABILITIES ARE:
    BIDIR PROBS
    NUMBER OF LINKS PER PAGE PROB
    WORD COUNT BASED PROB
    
    USE TRELLO TO ORGANIZE EVERTYHING
    PLAN NEXT STEPS
    MELHOR FEITO DO QUE PERFEITO
    
    

In [1]:
from wikipydia import dict_storage, wikidb, parse, wikisyn
WikisynDict = wikisyn.WikisynDict
from collections import Counter
from tabulate import tabulate
import re

In [18]:
def print_data(headers, data_list):
    """Function to print data tabulated """
    return print(tabulate(data_list, headers=headers))
#print_data(['seedrank', 'pagerank', 'in_degree', 'ufauisfha', 'isfudnifsu'], [[1,2,3,4]])

In [19]:
wiki_db = wikidb.WikiDb()

In [20]:
def print_list(iterable):
    for item in iterable:
        print(item)

In [32]:
arti, _ = wiki_db.get_article_by_href("MQTT")
MUST CREATE METHOD TO REMOVE REFERENCES AND EXTERNAL LINKS 
DATABASE MUST STORE ONLY A TUPLE WITH DATA AND ONCE REQUEST CREATE OBJECT ON THE FLY AND RETURN

print()


<p><b>MQTT</b><sup class="reference" id="cite_ref-1"><a href="#cite_note-1">[1]</a></sup> (<b>MQ Telemetry Transport</b> or <b>Message Queue Telemetry Transport</b>) is an ISO standard (ISO/IEC PRF 20922)<sup class="reference" id="cite_ref-ISO_2-0"><a href="#cite_note-ISO-2">[2]</a></sup> <a href="/wiki/Publish%E2%80%93subscribe_pattern" title="Publish–subscribe pattern">publish-subscribe</a>-based "lightweight" messaging protocol for use on top of the <a class="mw-redirect" href="/wiki/TCP/IP" title="TCP/IP">TCP/IP protocol</a>. It is designed for connections with remote locations where a "small code footprint" is required or the network bandwidth is limited. The <a href="/wiki/Publish%E2%80%93subscribe_pattern" title="Publish–subscribe pattern">publish-subscribe messaging pattern</a> requires a <a href="/wiki/Message_broker" title="Message broker">message broker</a>. The <a class="mw-redirect" href="/wiki/Broker_(service-oriented_architecture)" title="Broker (service-oriented archit

### Abstract Links only
Get only the abstract links (first section) and set the same proportional score for all of them

In [21]:
def get_abstract_links(href):
    wikiart, _ = wiki_db.get_article_by_href(href)
    sections = parse._split_html_h2_sections(wikiart.html)
    abstract_links_with_text = parse._filter_wiki_links(parse._extract_html_links(sections[0]))
    
    abstract_links = set([href for href, text in abstract_links_with_text])
    
    all_score = 1 / len(abstract_links)
    
    abstract_links_scores = [(href, all_score) for href in abstract_links]
    
    return abstract_links_scores

In [26]:
abstract_links = get_abstract_links("Computer")
#print(len(abstract_links))
#print_list(abstract_links[:20])
print_data(['Article', 'Score'], abstract_links[:20])

Article                   Score
---------------------  --------
Computer_programming   0.037037
Peripheral             0.037037
Arithmetic             0.037037
Consumer_electronics   0.037037
Processing_element     0.037037
Microwave_oven         0.037037
Computer_program       0.037037
Remote_control         0.037037
Internet               0.037037
Mobile_device          0.037037
Digital_data           0.037037
World_War_II           0.037037
Personal_computer      0.037037
Boolean_algebra        0.037037
Data                   0.037037
Industrial_robot       0.037037
Abacus                 0.037037
Memory_(computers)     0.037037
Industrial_Revolution  0.037037
Smartphone             0.037037


### Simple link occurrence count
Just count the number of times some link occurs and score with it. 


There is a problem with this approach that each link may be different but will point to the same thing. In order to detect these links we have to download every one to verify if they point to the same think. 

This issue will appear in other approachs

In [28]:
def simple_link_count(href):
    wikiart, _ = wiki_db.get_article_by_href(href)
    links = wikiart.links()
    
    link_counter = Counter()
    
    for href, text in links:
        link_counter[href] += 1
        
    scores_sum = sum(link_counter.values())
        
    return list(map(lambda a: (a[0], a[1]/scores_sum), link_counter.most_common()))

#print_list(simple_link_count("Computer")[:20])
print_data(['Article', 'Score'], simple_link_count("Computer")[:20])

Article                                  Score
----------------------------------  ----------
International_Standard_Book_Number  0.0261661
Computer_programming                0.00682594
Computer_program                    0.00682594
Digital_object_identifier           0.00682594
Turing-complete                     0.00568828
Central_processing_unit             0.00455063
Microprocessor                      0.00455063
Konrad_Zuse                         0.00455063
Compiler                            0.00455063
Transistor                          0.00455063
Smartphone                          0.00455063
Vacuum_tube                         0.00455063
Mouse_(computing)                   0.00455063
Z3_(computer)                       0.00341297
MIPS_architecture                   0.00341297
James_Thomson_(engineer)            0.00341297
Electronics                         0.00341297
Control_flow                        0.00341297
Brian_Randell                       0.00341297
Peripheral    

### Count local wikisyn
Compute a wikisyn collection with the local links and score links with them.


The wikisyn class computes every href tag text (< a>text< /a>) with their links in order to find link-synonims, that is diferent texts that points to the same links.

Variations to this method may be the use of global wikisyn which computes links syns based on other pages references to the links present on the target article improving the accuracy.

In [86]:
def get_href_text_count(href):
    synDict = WikisynDict()
    
    wikiart, _ = wiki_db.get_article_by_href(href)
    links = wikiart.links()
    page_text = wikiart.text()
    
    #Populate syndict
    for href, text in links:
        synDict.add_link(href, text)
        
    #Get matches
    links_score = Counter()
        
    for link_href in synDict.keys():
        for l_text, l_score in synDict[link_href]:
            matches = re.findall('[^a-zA-Z0-9_]' + re.escape(l_text) + '[^a-zA-Z0-9_]', page_text, re.IGNORECASE)
            matches_score = len(matches) * l_score
            links_score[link_href] += matches_score

    scores_sum = sum(links_score.values())
            
    norm_links_scores = list(map(lambda a: (a[0],synDict[a[0]], a[1]/scores_sum), links_score.most_common()))
            
    return norm_links_scores

print_list(get_href_text_count("Computer")[:20]) 

('Logical_conjunction', [('AND', 1.0)], 0.12580914181327188)
('Logical_disjunction', [('OR', 1.0)], 0.032405385012509426)
('Machine', [('machine', 1.0)], 0.02763988721655216)
('Memory_(computers)', [('memory', 1.0)], 0.02287438942059489)
('Computer_memory', [('memory', 1.0)], 0.02287438942059489)
('Computer_program', [('program', 0.5), ('computer programs', 0.16666666666666666), ('Computer program', 0.16666666666666666), ('programs', 0.16666666666666666)], 0.017632341845041893)
('Instruction_(computer_science)', [('instructions', 1.0)], 0.017632341845041893)
('Computing', [('computing', 1.0)], 0.01667924228585044)
('Computer_data_storage', [('memory', 0.6666666666666666), ('Computer data storage', 0.3333333333333333)], 0.01540844287359517)
('Electronics', [('electronic', 1.0)], 0.014773043167467532)
('Data_(computing)', [('data', 0.5), ('Data', 0.5)], 0.012390294269488898)
('Digital_data', [('digital', 1.0)], 0.011913744489893172)
('Digital_electronics', [('digital', 1.0)], 0.011913744

In [ ]:
def get_links_score(page):
    """Function to cross a list of links with a text, setting scores."""
    
    wikiart = get_wiki_article_by_href(page)
    
    pageid = wikiart.page_id
    #Ensure only one href is present
    page_links = set([link_href for link_href, link_text in wikiart.links()])
    page_text = wikiart.text()
    
    links_score = dict()
    
    norm_fact = 0 #norm factor to results sum to one
    
    #Ensure all page links are present
    #check_and_download(page_links)
    
    #The ideia is to get for each href the texts that use to follow these hrefs in wikipedia articles.
    #The more each term appears, it got increased weight.
    #Then we try to match each of the terms for each href to the wiki text, applying to each match the correspondent weight.
    #The weight stuff is good to avoid cases where the href_text have appeared only once in one article,
    #but it is a frequent term in other articles but offen does not mean the original href it pointed to
    
    for link_href in page_links:
        links_score[link_href] = 0
        for l_text, l_score in synbeta.get_synoms(link_href): #get_all_wikisyns_by_href(link_href): #wikisyn[link_href]:
            matches = re.findall('[^a-zA-Z0-9_]' + re.escape(l_text) + '[^a-zA-Z0-9_]', page_text, re.IGNORECASE)
            matches_score = len(matches) * l_score
            links_score[link_href] += matches_score
            norm_fact += matches_score
            
    norm_links_score = dict(map(lambda a: [a[0], float(a[1])/norm_fact], links_score.items()))
            
    return norm_links_score

#v_sum = 0
links_score = get_links_score("Convolutional neural network")
for k, v in sorted(links_score.items(), key=lambda a:a[1], reverse=True):
    print(k,synbeta.hrefs[k],v)
    #v_sum += v
#print v_sum

In [37]:
get_href_text_count("MQTT")

defaultdict(<class 'collections.Counter'>, {'Publish%E2%80%93subscribe_pattern': Counter({'publish-subscribe': 1, 'publish-subscribe messaging pattern': 1}), 'TCP/IP': Counter({'TCP/IP protocol': 1, 'TCP/IP': 1}), 'Message_broker': Counter({'message broker': 1}), 'Broker_(service-oriented_architecture)': Counter({'broker': 1}), 'Messaging_pattern': Counter({'messages': 1}), 'Andy_Stanford-Clark': Counter({'Andy Stanford-Clark': 1}), 'OASIS_(organization)': Counter({'OASIS': 1}), 'ZigBee': Counter({'ZigBee': 1}), 'Message_queuing': Counter({'message queuing': 1}), 'Advanced_Message_Queuing_Protocol': Counter({'Advanced Message Queuing Protocol': 1}), 'Constrained_Application_Protocol': Counter({'Constrained Application Protocol': 1}), 'XMPP': Counter({'XMPP': 1}), 'Web_Application_Messaging_Protocol': Counter({'Web Application Messaging Protocol (WAMP)': 1}), 'Facebook_Messenger': Counter({'Facebook Messenger': 1}), 'Online_chat': Counter({'online chat': 1}), 'EVRYTHNG': Counter({'EVRYT